In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
import json
import geopandas as gpd


RTA_ROUTE_URL = 'https://opendata.arcgis.com/datasets/1cb5c63d6f114f8a94c6d5a0e03ae62e_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D'
RTA_ROUTE_DATA = requests.get(RTA_ROUTE_URL)

RTA_STOPS_URL = 'https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/RTAs/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json'
RTA_STOPS_DATA = requests.get(RTA_STOPS_URL)


# Rishab helper code
def add_census_tract(dataframe):
    polygons = gpd.read_file("data/tl_2019_25_tract/tl_2019_25_tract.shp")
    polygons = polygons.rename(columns={"TRACTCE": "census_tract"}, index=str)
    polygons = polygons.to_crs("EPSG:26986")
    gdf = dataframe
    df = gpd.sjoin(gdf, polygons[['census_tract', 'geometry']], how='left', op='within')
    df.drop(columns=['index_right'], inplace=True)
    return df


def get_median_hh_income():
    '''
    Returns Pandas DataFrame representation Median Household Income Estimate by Census Tract for MA.
    American Community Survey (ACS) 2018 Census data used.
    Specific table: ACS 2018 5-year detailed table "B19013_001E"
    '''
    URL = "https://api.census.gov/data/2018/acs/acs5?get=B19013_001E&for=tract:*&in=state:25"

    response = requests.get(url = URL)
    data = response.json()

    median_income_df = pd.DataFrame(data[1:len(data)-1], columns = data[0])

    return median_income_df

In [2]:
route_df = pd.read_csv(StringIO(RTA_ROUTE_DATA.content.decode()))
stops_data = json.loads(RTA_STOPS_DATA.content)['features']

# Get only brockton
route_df = route_df.loc[route_df['Agency'] == 'BrocktonAreaRTA']
print(route_df)

    OBJECTID  shape_id agency_id route_id route_short_name  \
0          1    p_6097       444     2933                1   
1          2    p_6101       444     2934                2   
2          3    p_6102       444     2935                3   
3          4    p_6103       444     2935                3   
4          5    p_6105       444     2940                8   
5          6    p_6108       444     2946               14   
6          7    p_6110       444     2938                5   
7          8    p_6113       444     2942               10   
8          9    p_6115       444     3219               12   
9         10    p_6117       444     2933                1   
10        11    p_6118       444     2934                2   
11        12    p_6119       444     2935                3   
12        13    p_6120       444     2935                3   
13        14    p_6124       444     2936                4   
14        15    p_6125       444     2938                5   
15      

In [3]:
print(json.dumps(stops_data[:20], indent=2))

[
  {
    "attributes": {
      "OBJECTID": 1,
      "stop_id": "2537977",
      "stop_code": "810.0",
      "platform_code": null,
      "stop_name": "Market Basket",
      "stop_desc": null,
      "stop_lat": 42.09394241,
      "stop_lon": -71.05217796,
      "zone_id": "863.0",
      "stop_url": null,
      "location_type": "0",
      "parent_station": null,
      "stop_timezone": "America/New_York",
      "position": null,
      "direction": null,
      "wheelchair_boarding": "0",
      "Agency": "BrocktonAreaRTA"
    },
    "geometry": {
      "x": -71.05217795899995,
      "y": 42.09394241000007
    }
  },
  {
    "attributes": {
      "OBJECTID": 2,
      "stop_id": "2558590",
      "stop_code": null,
      "platform_code": null,
      "stop_name": "Belair St at 7-Eleven",
      "stop_desc": null,
      "stop_lat": 42.08901924,
      "stop_lon": -71.03720398,
      "zone_id": null,
      "stop_url": null,
      "location_type": "0",
      "parent_station": null,
      "stop_time

In [4]:
get_median_hh_income()

,B19013_001E,state,county,tract
0,132750,25,025,060501
1,12759,25,025,061101
2,84083,25,025,070101
3,28851,25,025,070402
4,52676,25,025,071101
...,...,...,...,...
1472,69750,25,005,613902
1473,52741,25,005,630101
1474,58362,25,005,630102
1475,34692,25,005,640901
